In [1]:

# rm(list = ls(all.names = TRUE)) # will clear all objects
# gc() # free up memory and report the memory usage

library(tidyverse)
library(readxl)
library(openxlsx)
library(xts)
library(stringi)
library(lubridate)
library(dynlm)
library(vars)
library(plotly)
library(dplyr)
library(zoo)
library(ggplot2)
library(ggfortify)
library(ggpubr)


# start date of the data and end date for data and forecasting in the imported csv file (check the file first)
datestart.data <- "2020/1/22"   ### Start of past date
dateend.data <- "2021/10/06"      ### End of past date
datestart.fcst <- "2021/10/07"    ### Start of forecast date
dateend.fcst <- "2023/12/31"    ### End of forecast date
date.data <- seq(as.Date(datestart.data), as.Date(dateend.fcst),"days")
date.fcst <- seq(as.Date(datestart.fcst), as.Date(dateend.fcst),"days")
date.full <- seq(as.Date(datestart.data), as.Date(dateend.fcst),"days")

date.full.wd <- wday(date.full)

dd.Mon <- ifelse(date.full.wd==2, 1, 0)
dd.Tue <- ifelse(date.full.wd==3, 1, 0)
dd.Wed <- ifelse(date.full.wd==4, 1, 0)
dd.Thu <- ifelse(date.full.wd==5, 1, 0)
dd.Fri <- ifelse(date.full.wd==6, 1, 0)
dd.Sat <- ifelse(date.full.wd==7, 1, 0)
dd.Sun <- ifelse(date.full.wd==1, 1, 0)

setwd("/home/sysadm/Desktop/pogba_backup2/0_Gil_Prediction/02_code/VAR/R/211105")
# Import the csv data
df <- read.csv("../../../../01_data/VAR/var_211105.csv")   #### Update csv file
df <- cbind(df,dd.Mon,dd.Tue,dd.Wed,dd.Thu,dd.Fri,dd.Sat,dd.Sun)


df <- df %>% 
  mutate(Confirmed.MA7 = rollmean(Confirmed, k = 7, fill = 0, align = "right")) %>%
  mutate(Deaths.MA7 = rollmean(Deaths, k = 7, fill = 0, align = "right")) %>%
  mutate(Severity.MA7 = rollmean(Severity, k = 7, fill = 0, align = "right")) 


# select the endogenous variables in y
y.df <- select(df,Confirmed.MA7,Deaths.MA7,Severity.MA7)
y.xts <- as.xts(y.df, order.by = date.data)

# select the exogenous variables in x
x.df <- select(df,VM_alpha,VM_delta,d_Vaccine)
x.xts <- as.xts(x.df, order.by = date.data)

# select weekday dummies (excluding Sunday)
wd.df <- select(df,dd.Mon,dd.Tue,dd.Wed,dd.Thu,dd.Fri,dd.Sat)
wd.xts <- as.xts(wd.df, order.by = date.data)

# select social distance varaible
OX.df <- select(df,Ox)
OX.xts <- as.xts(OX.df, order.by = date.data)

# sample size and forecasting window
size.H <- length(date.fcst)         # forecasting window: in this case, the length of future values in x
size.T <- length(date.data)-size.H  # sample size for estimation

n <- ncol(y.df) # Number of endogenous variables (y) 
k <- ncol(x.df) # Number of exogenous variables (x) 

## VAR-X estimation ------------------------------------------------------------------------------

x.xts.est <- x.xts[7:size.T,] 
y.xts.est <- y.xts[7:size.T,]
wd.xts.est <- wd.xts[7:size.T,]
OX.xts.est <- OX.xts[7:size.T,]

VAR.exogen <- cbind(OX.xts.est)

── Attaching packages ─────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.5     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.0.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



Attaching package: ‘xts’


The following objects are masked from ‘package:dplyr’:

    first, last



Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


Loading required package: MASS


Attaching package: ‘MASS’


The following object is masked from ‘package:dplyr’:

    select


Loading required package: strucc

In [2]:
# Lag selection (including x into the VAR)
VAR.xts.est <- cbind(y.xts.est,x.xts.est)
# VAR.xts.est <- na.omit(VAR.xts.est)
VAR.select <- VARselect(VAR.xts.est, lag.max = 10, type = "const", exogen =VAR.exogen) # Lag selection
VAR.select

,1,2,3,4,5,6,7,8,9,10
AIC(n),-1.358777e+01,-1.612932e+01,-1.612248e+01,-1.610520e+01,-1.611857e+01,-1.612180e+01,-1.630700e+01,-1.936392e+01,-1.993504e+01,-1.995493e+01
HQ(n),-1.345231e+01,-1.589227e+01,-1.578384e+01,-1.566497e+01,-1.557674e+01,-1.547839e+01,-1.556200e+01,-1.851732e+01,-1.898685e+01,-1.890515e+01
SC(n),-1.323960e+01,-1.552002e+01,-1.525205e+01,-1.497364e+01,-1.472588e+01,-1.446799e+01,-1.439206e+01,-1.718785e+01,-1.749784e+01,-1.725660e+01
FPE(n),1.255771e-06,9.888899e-08,9.957690e-08,1.013297e-07,1.000113e-07,9.972761e-08,8.291225e-08,3.902350e-09,2.206412e-09,2.165396e-09


In [3]:
opt.p <- VAR.select$selection[3] # the opt.p (= optimal p) is determined by the AIC in VAR.select
VAR.trial <- VAR(y=VAR.xts.est,p = opt.p, type = "const",exogen =VAR.exogen)

VAR.trial


VAR Estimation Results:

Estimated coefficients for equation Confirmed.MA7: 
Call:
Confirmed.MA7 = Confirmed.MA7.l1 + Deaths.MA7.l1 + Severity.MA7.l1 + VM_alpha.l1 + VM_delta.l1 + d_Vaccine.l1 + Confirmed.MA7.l2 + Deaths.MA7.l2 + Severity.MA7.l2 + VM_alpha.l2 + VM_delta.l2 + d_Vaccine.l2 + Confirmed.MA7.l3 + Deaths.MA7.l3 + Severity.MA7.l3 + VM_alpha.l3 + VM_delta.l3 + d_Vaccine.l3 + Confirmed.MA7.l4 + Deaths.MA7.l4 + Severity.MA7.l4 + VM_alpha.l4 + VM_delta.l4 + d_Vaccine.l4 + Confirmed.MA7.l5 + Deaths.MA7.l5 + Severity.MA7.l5 + VM_alpha.l5 + VM_delta.l5 + d_Vaccine.l5 + Confirmed.MA7.l6 + Deaths.MA7.l6 + Severity.MA7.l6 + VM_alpha.l6 + VM_delta.l6 + d_Vaccine.l6 + Confirmed.MA7.l7 + Deaths.MA7.l7 + Severity.MA7.l7 + VM_alpha.l7 + VM_delta.l7 + d_Vaccine.l7 + Confirmed.MA7.l8 + Deaths.MA7.l8 + Severity.MA7.l8 + VM_alpha.l8 + VM_delta.l8 + d_Vaccine.l8 + Confirmed.MA7.l9 + Deaths.MA7.l9 + Severity.MA7.l9 + VM_alpha.l9 + VM_delta.l9 + d_Vaccine.l9 + const + Ox 

Confirmed.MA7.l1    Dea

In [4]:

x.xts.exg <- lag.xts(x.xts,1:opt.p)
x.xts.exg.est <- x.xts.exg[7:size.T,]

VAR.exog <- cbind(x.xts.exg.est,OX.xts.est)

VAR.est <- VAR(y = y.xts.est, p = opt.p,type = "const",exogen = VAR.exog)
VAR.est


VAR Estimation Results:

Estimated coefficients for equation Confirmed.MA7: 
Call:
Confirmed.MA7 = Confirmed.MA7.l1 + Deaths.MA7.l1 + Severity.MA7.l1 + Confirmed.MA7.l2 + Deaths.MA7.l2 + Severity.MA7.l2 + Confirmed.MA7.l3 + Deaths.MA7.l3 + Severity.MA7.l3 + Confirmed.MA7.l4 + Deaths.MA7.l4 + Severity.MA7.l4 + Confirmed.MA7.l5 + Deaths.MA7.l5 + Severity.MA7.l5 + Confirmed.MA7.l6 + Deaths.MA7.l6 + Severity.MA7.l6 + Confirmed.MA7.l7 + Deaths.MA7.l7 + Severity.MA7.l7 + Confirmed.MA7.l8 + Deaths.MA7.l8 + Severity.MA7.l8 + Confirmed.MA7.l9 + Deaths.MA7.l9 + Severity.MA7.l9 + const + VM_alpha + VM_delta + d_Vaccine + VM_alpha.1 + VM_delta.1 + d_Vaccine.1 + VM_alpha.2 + VM_delta.2 + d_Vaccine.2 + VM_alpha.3 + VM_delta.3 + d_Vaccine.3 + VM_alpha.4 + VM_delta.4 + d_Vaccine.4 + VM_alpha.5 + VM_delta.5 + d_Vaccine.5 + VM_alpha.6 + VM_delta.6 + d_Vaccine.6 + VM_alpha.7 + VM_delta.7 + d_Vaccine.7 + VM_alpha.8 + VM_delta.8 + d_Vaccine.8 + Ox 

Confirmed.MA7.l1    Deaths.MA7.l1  Severity.MA7.l1 Confi

In [ ]:


################# Confirmed #####################################################################
## Forecasting with VAR-X ------------------------------------------------------------------------

x.xts.fcst <- x.xts.exg[(size.T+1):(size.T+size.H),]
wd.xts.fcst <- wd.xts[(size.T+1):(size.T+size.H),]
OX.xts.fcst <- OX.xts[(size.T+1):(size.T+size.H),]
#x.xts.exg.trial <- x.xts[(size.T+1):(size.T+size.H),]

fcst.exog <- cbind(x.xts.fcst,OX.xts.fcst)
fcst.trial <- cbind(OX.xts.fcst)

y.fcst <- predict(VAR.est, n.ahead = size.H, ci = 0.95, dumvar = fcst.exog)

fcst.Confirmed <- as.xts(y.fcst$fcst$Confirmed.MA7,order.by = date.fcst)
forecast <- as.data.frame(fcst.Confirmed)
names(forecast) <- c("PointForecast", "Lo95", "Hi95")
write.csv(forecast,file='../../../../03_result/SWAR/211105/swar_211105_i.csv')

fcst.Deaths <- as.xts(y.fcst$fcst$Deaths.MA7,order.by = date.fcst)
forecast <- as.data.frame(fcst.Deaths)
names(forecast) <- c("PointForecast", "Lo95", "Hi95")
write.csv(forecast,file='../../../../03_result/SWAR/211105/swar_211105_d.csv')

fcst.Severity <- as.xts(y.fcst$fcst$Severity.MA7,order.by = date.fcst)
forecast <- as.data.frame(fcst.Severity)
names(forecast) <- c("PointForecast", "Lo95", "Hi95")
write.csv(forecast,file='../../../../03_result/SWAR/211105/swar_211105_s.csv')

plot.xts(fcst.Confirmed[,1:3],plot.type = "single")
plot.xts(fcst.Deaths[,1:3],plot.type = "single")
plot.xts(fcst.Severity[,1:3],plot.type = "single")